Student ID: 3034963465

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import random
import copy

In [ ]:
import sys
!{sys.executable} -m pip install mat73
import mat73


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
datadir = "/content/gdrive/MyDrive/NeuralComputation/"
dataset_2D = mat73.loadmat(datadir+'PS3_2D_datasets.mat')
dataset_bars = mat73.loadmat(datadir+'PS3_bars_dataset.mat')
dataset_faces = mat73.loadmat(datadir+'PS3_faces_dataset.mat')



# **Part 1: PCA and linear Hebbian learning**

In [ ]:

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10, 10))
fig.suptitle('Data')

ax1.scatter(dataset_2D['gaussian'][0], dataset_2D['gaussian'][1])
ax1.set_xlabel('X1')
ax1.set_ylabel('X2')
ax1.set_title('Gaussian')

ax2.scatter(dataset_2D['nongaussian'][0], dataset_2D['nongaussian'][1])
ax2.set_xlabel('X1')
ax2.set_ylabel('X2')
ax2.set_title('Non-gaussian')



i) gaussian data

In [ ]:
gaussian_data = dataset_2D['gaussian']
training_iterations = 1500
n_neurons = 2 # how many neurons 
learningRate = 0.0001 # should be really small 
weights = np.random.randn(2,2) # initialize weights


In [ ]:
def sanger_learning_rule(x,w,lr,n_neurons):

    y = w.T @ x # w * x, but for matrix muliplication     
    x1 = x
    delta_w = np.zeros(w.shape)
    
    for i in range(n_neurons):
        
        x1 = x1 - (w[:,i,None] @ y[None,i,:])
        delta_w[:,i] = x1 @ y[i,:].T

    w += delta_w*lr 
    
    return w

In [ ]:
ws = np.zeros((2,2))
all_weights = np.tile(ws, (training_iterations,1,1))

for i in range(training_iterations):
  weights = sanger_learning_rule(gaussian_data,weights,learningRate,n_neurons)
  all_weights[i]= weights

weights


In [ ]:

indices = [0,int(training_iterations/2),training_iterations-1]
idx = 2
idx_to_plot = indices[idx]
titles = ['Initial','Partway','Final']


fig = plt.subplots(figsize=(8, 8))
plt.scatter(gaussian_data[0],gaussian_data[1])
plt.xlabel('X1')
plt.ylabel('X2')
plt.title(titles[idx])
for i in range(n_neurons):
    plt.plot([0,all_weights[idx_to_plot][0,i]],[0,all_weights[idx_to_plot][1,i]],'k',linewidth=4,alpha=0.8);


ii) non-gaussian data

In [ ]:
non_gaussian_data = dataset_2D['nongaussian']
training_iterations = 1500 # has to be really high
n_neurons = 2 # how many neurons 
learningRate = 0.0001 # should be really small 
weights = np.random.randn(2,2) # initialize weights


ws = np.zeros((2,2))
all_weights = np.tile(ws, (training_iterations,1,1))

for i in range(training_iterations):
  weights = sanger_learning_rule(non_gaussian_data,weights,learningRate,n_neurons)
  all_weights[i]= weights



In [ ]:
indices = [0,int(training_iterations/2),training_iterations-1]
idx = 2
idx_to_plot = indices[idx]
titles = ['Initial','Partway','Final']

fig = plt.subplots(figsize=(8, 8))
plt.scatter(non_gaussian_data[0],non_gaussian_data[1])
plt.xlabel('X1')
plt.ylabel('X2')
plt.title(titles[idx])
for i in range(n_neurons):
    plt.plot([0,all_weights[idx_to_plot][0,i]],[0,all_weights[idx_to_plot][1,i]],'k',linewidth=4,alpha=0.8);


In [ ]:
# synthesize data 
y = weights.T @ non_gaussian_data

pc1 = np.random.normal(y[0].mean(), np.sqrt(y[0].var()), 1000)
pc2 = np.random.normal(y[1].mean(), np.sqrt(y[1].var()), 1000)
gaussians = np.array([pc1,pc2])

synthesize = weights.T @ gaussians

figure = plt.subplots(figsize=[5,5])
plt.scatter(synthesize[0],synthesize[1])
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('data synthesis')

iii) faces

In [ ]:
# Plot subset of original faces from the dataset 


numfaces=len(dataset_faces['faces'])
num_inputs_per_face=len(dataset_faces['faces'][0].flatten())
subset_faces = [6,8,10] #random images to be plotted
face_data_subset=np.empty([len(subset_faces), num_inputs_per_face]) 


for i in range(len(subset_faces)):
  face_data_subset[i]=dataset_faces['faces'][subset_faces[i]].flatten()
  plt.subplot(1,len(subset_faces),i+1)
  plt.imshow(face_data_subset[i].reshape(37,28))



In [ ]:
# train sanger rule 

def sanger_Learn_Faces(x,weights,lr):
    
    y = weights.T @ x     
    x1 = x
    delta_weight = np.zeros(weights.shape)
    
    for i in range(y.shape[0]):
        
        x1 = x1 - (weights[:,i,None] @ y[None,i,:])
        delta_weight[:,i] = x1 @ y[i,:].T

    weights += delta_weight*lr 
    return weights

face_data=np.empty([numfaces, num_inputs_per_face]) 

for i in range(len(face_data)):
  face_data[i]=dataset_faces['faces'][i].flatten()

face_data = face_data - np.atleast_2d(np.mean(face_data,1)).T


dataset = face_data.T
numOutputs = 5
numInputs,numDatapoints = dataset.shape
learningRate = 1e-9#1e-10
weights = np.random.random([numInputs,numOutputs]) 
numSteps = 500

for i in range(numSteps):
    weights = sanger_Learn_Faces(dataset,weights,learningRate)


In [ ]:
# show network encoding 
plt.figure(figsize=(20,6))

for i in range(numOutputs):
    plt.subplot(1,numOutputs,i+1)
    plt.imshow(weights[:,i].reshape(37,28))


In [ ]:
face_data=np.empty([numfaces, num_inputs_per_face]) 
for i in range(len(face_data)):
  face_data[i]=dataset_faces['faces'][i].flatten() 
div=1
face_avg = np.mean(face_data,0)
face_idx = 10
selected_face = dataset[:,face_idx,None]
y = weights.T/div @ selected_face
reconstructed = (weights/div @ y)+face_avg[:,None] 
plt.imshow(reconstructed.reshape(37,28))


synthesis

In [ ]:
num_imgs = 10

fig, ax = plt.subplots(2,5,figsize=(18,10))

for i in np.arange(num_imgs):   
    
    n_neurons = 5
    random_input = np.random.uniform(1,256, size=[1036,1])
    synthesized_img = np.ones([1036,1])
    
    for n in np.arange(n_neurons):
        y    = np.dot(random_input.T,weights[:,n,None])
        y_sample = np.random.normal(0,np.sqrt(np.abs(y)))
        synthesized_img += weights[:,n,None]*y_sample

    ax[(i >= 5)*1, i%5].imshow(synthesized_img.reshape([37,28]))
   

plt.show()

# **Part 2: Winner-take-all**

In [ ]:
def Winner_take_all_learning(x,w,lr,n_neurons):
    
    y = w @ x 
    win_idx = np.argmax(y,axis=0) 
    
    for i in range(n_neurons):
        win_mask = np.equal(i,win_idx)
        w[i,:,None] += lr*((x - w [i,:,None]) @ win_mask[:,None])
        
    return weights

n_neurons = 4
learning_rate = 0.004
data = non_gaussian_data
weights = np.random.random([n_neurons,data.shape[0]]) 



for i in range(300):
  weights = Winner_take_all_learning(data,weights,learning_rate,n_neurons)


fig = plt.subplots(figsize=(8, 8))
plt.scatter(non_gaussian_data[0],non_gaussian_data[1])
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('wta')

for i in range(n_neurons):
    plt.plot([0,weights[i,0]],[0,weights[i,1]],'k',linewidth=4,alpha=0.8);



ii) implement wta with faces

In [ ]:
def Winner_take_all_learning_faces(dataset,weights,lr):

    y = weights.T @ dataset
    delta_w = np.zeros(weights.shape)
    win_idx = np.argmax(y,axis=0) 

    for i in range(y.shape[0]):
        delta_w[:,i,None] = lr*((dataset - weights [:,i,None]) @ np.equal(i,win_idx)[:,None])
    
    weights += delta_w
    return weights


face_data=np.empty([numfaces, num_inputs_per_face]) 

for i in range(len(face_data)):
  face_data[i]=dataset_faces['faces'][i].flatten() #-np.mean(dataset_faces['faces'][i].flatten())
face_data = face_data - np.atleast_2d(np.mean(face_data,1)).T

dataset = face_data.T
n_neurons = 40
n_inputs_per_image,n_faces = dataset.shape
lr =.17
weights = np.random.randn(n_inputs_per_image,n_neurons) 

numSteps = 75#150 & 0.12 or .11 and something idk 
# reduceFactor = .01
for i in range(numSteps):
    weights = Winner_take_all_learning_faces(dataset,weights,lr)
    # reduceFactor += .000001 # in case we want to reduce learning rate as training goes, but doesn't really seem to help.


plt.figure(figsize=(20,8))
for i in range(n_neurons):
    plt.subplot(4,n_neurons//4+n_neurons%2,i+1)
    neuronimg = weights[:,i]
    neuronimg = neuronimg.reshape(37,28)
    plt.imshow(neuronimg)


In [ ]:
# reconstruction 
face_data=np.empty([numfaces, num_inputs_per_face]) 
for i in range(len(face_data)):
  face_data[i]=dataset_faces['faces'][i].flatten() 
face_avg = np.mean(face_data,0)
face_idx =8

fws=weights

plt.figure(figsize=(20,8))

for i in range(n_neurons):
    
    y = fws[:,0:i+1].T @ selected_face

    reconstructed = (fws[:,0:i+1] @ y)+face_avg[:,None] 

    plt.subplot(4,n_neurons//4+n_neurons%2,i+1)
    plt.imshow(reconstructed.reshape(37,28))
    


# **Part 3: Foldiak sparse representations**

In [ ]:
def foldiak_rule(x, q, w, t, numtrials):


    y = np.zeros((q.shape[1], x.shape[1])) 
    get_output = np.zeros((q.shape[1], x.shape[1]))
    unit_activations = q.T @ x
    
    for trial in range(numtrials):
        delta_y = sigmoid(unit_activations + w.T @ y - t[:, None],1) - y
        y += delta_t * delta_y

    # according to foldiak paper we need to look for values greater than 0.5 
    get_output[np.where(y>0.5)] = 1
    
    return get_output



def foldiak_weight_updates(x, y, q, w, t, alpha, beta, gamma):
    act = np.mean(y, axis=1)
    between_unit_correlation = y @ y.T / x.shape[1]
    input_output_correlation = x @ y.T / x.shape[1]
    
    # update lateral weights and remove the diagonals (where i == j ) so that the units don't inhibit themselves, but just other neurons. 
    w +=  -alpha * (between_unit_correlation - p**2)
    w -= np.diag(np.diag(w))
    w[np.where(w>0)] = 0
    
    #update threshold and forward weights
    q += beta * (input_output_correlation - q * act)
    t += gamma * (act - p)
    
    return (q, w, t)

In [ ]:
delta_t = 0.001 
numtrials = 1000 
numtrials_1 = 1000 

p = 1/8

#learning rates from the paper
alpha = 0.1 # lateral weights learning rate
beta = 0.05 # feedforward weights 
gamma = 0.1 # thresholds

n_neurons = 16

lambda_sigmoid = 10
numpixels = 8*8 # number of inputs/pixels per image

q = np.random.randn(numpixels, n_neurons) # feed forward weights
w = np.zeros((n_neurons, n_neurons)) # lateral weights 
t = np.ones((n_neurons)) # thresholds


def sigmoid(data,lambda_sigmoid):
    return 1.0 / (1.0 + np.exp(-lambda_sigmoid*data))

bars_data=np.empty([len(dataset_bars['bars']), numpixels]) 

for i in range(len(dataset_bars['bars'])):
  bars_data[i] = dataset_bars['bars'][i].flatten()

bardata = bars_data.T
for trial in range(numtrials):
    print(trial)
    
    output = foldiak_rule(bardata, q, w, t, numtrials_1)    
    (q, w, t) = foldiak_weight_updates(bardata, output, q, w, t, alpha, beta, gamma)
   


In [ ]:
fig, axs = plt.subplots(2, 8, figsize=(12, 8))
fig.suptitle('Feedforward weights')

for i in range(n_neurons):

    fw_weights=q[:,i]

    plt.subplot(2,n_neurons//2+n_neurons%2,i+1)
    plt.imshow(fw_weights.reshape(8,8),cmap='gray')




In [ ]:
fig, axs = plt.subplots(figsize=(12, 8))
plt.bar(np.arange(t.shape[0]),t)
plt.ylabel('thresholds')
plt.xlabel('neurons')


In [ ]:
fig, axs = plt.subplots(figsize=(8, 8))
plt.imshow(w,cmap='gray')
plt.title('lateral weights')